<a href="https://colab.research.google.com/github/raquelcarmo/tropical_cyclones/blob/import-py-files/src/code/TC_Category_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tropical Cyclones Categorization

Script to train Deep Learning models to categorize TCs based on their topology patterns.

##Imports and configurations

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/tropical_cyclones/src/code
%ls

import imp 
# import helper.py
helper = imp.new_module('helper_functions')
exec(open("./helper_functions.py").read(), helper.__dict__)
# import models.py
models = imp.new_module('models')
exec(open("./models.py").read(), models.__dict__)
# import data_processor.py
dp = imp.new_module('data_processor')
exec(open("./data_processor.py").read(), dp.__dict__)

In [ ]:
# insert your desired path to work on
%cd /content/drive/My Drive/ESRIN_PhiLab/Tropical_Cyclones/data
%ls

In [ ]:
# general imports
import random
import glob
import os
import sys
sys.stdout.flush()
import pandas as pd
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import math
import imageio
import os.path
import time
from PIL import Image
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import ndimage
from google.colab.patches import cv2_imshow
import random
from shapely.geometry import Point
import re
import pickle
import scipy
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import datetime

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.data import Dataset
from tensorflow.keras import Input
from tensorflow.keras.applications import resnet50, mobilenet_v2, vgg16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras.layers import concatenate, Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import CategoricalAccuracy, TopKCategoricalAccuracy, Precision, Recall, TruePositives, FalsePositives, TrueNegatives, FalseNegatives
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

np.set_printoptions(precision=4)

## 1.Train on data according to csv split into train, val and test sets

Prepare the tf.data.Dataset instances to be fed to the model.

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet"]
MIN_HEIGHT = 288
MIN_WIDTH = 288
NUM_VARS = False
NORMALISE = True
ROTATE = False
BATCH_SIZE = 8
BUFFER_SIZE = 100
EPOCHS = 20
LEARNING_RATE = 0.0001
##################################

# load data
train_images, train_labels, train_bbox = helper.load_data("{}/csv/training.csv".format(main_dir))
val_images, val_labels, val_bbox = helper.load_data("{}/csv/val.csv".format(main_dir))
test_images, test_labels, test_bbox = helper.load_data("{}/csv/test.csv".format(main_dir))
class_weights = helper.compute_class_weights("{}/csv/full_dataset.csv".format(main_dir))


# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                             min_height = MIN_HEIGHT,
                             min_width = MIN_WIDTH,
                             normalise = NORMALISE,           # perform normalisation
                             rotate = ROTATE,                 # perform rotation
                             plot_light = False,              # plot only select_crop() images
                             plot_extensive = False,          # plot extensively all images
                             show_prints = False)


# generate datasets
train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
test_dataset = helper.prepare_dataset(processor, test_images, test_labels, test_bbox)
#for image, label in train_dataset:
#  plt.imshow(image)
#  plt.show()
#  print("FINAL - image: {}, label: {}".format(image.shape, label))
#  print("FINAL - image: {}, max: {}, min: {}, label: {}".format(image.shape, np.max(image), np.min(image),  label))

train_dataset, val_dataset = helper.z_norm(train_dataset, val_dataset)
#cnt = 0
#for image, label in train_dataset:
#  cnt +=1
#  plt.imshow(image)
#  plt.show()

# configure for performance
train_dataset = helper.configure_for_performance(train_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True, augment = True)
val_dataset = helper.configure_for_performance(val_dataset, BUFFER_SIZE, BATCH_SIZE)
test_dataset = helper.configure_for_performance(test_dataset, BUFFER_SIZE, BATCH_SIZE)

Script to perform end-to-end training in the model.

In [ ]:
# directory to save results
dir = '{}_Numeric-{}_BatchSize-{}_Epochs-{}_Norm-{}'.format(NETWORK, str(NUM_VARS)[0], str(BATCH_SIZE), str(EPOCHS), str(NORMALISE)[0])
save_dir = 'classification_results/categorization/' + dir + '/'
os.makedirs(save_dir, exist_ok=True)

# MAKE MODEL
if NETWORK == "Mobile":
    model = models.make_MobileNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE)
elif NETWORK == "ResNet":
    model = models.make_ResNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE)
else:
    sys.exit("Incert valid network model. Options: Mobile or ResNet (case sensitive)")

# CREATE CALLBACKS
callbacks = [
    EarlyStopping(patience = 7, verbose = 1),
    ReduceLROnPlateau(factor = 0.1, patience = 5, min_lr = 0.00001, verbose = 1),
    ModelCheckpoint(save_dir + "best_model.h5", verbose = 1, save_best_only = True),
    TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
]

# TRAIN THE NETWORK
H = model.fit(
    train_dataset,
    steps_per_epoch = len(train_dataset),
    validation_data = val_dataset,
    validation_steps = len(val_dataset),
    epochs = EPOCHS,
    callbacks = callbacks,
    verbose = 1,
    class_weight = class_weights,
    shuffle = True)

In [ ]:
# Load the TensorBoard notebook extension
#%load_ext tensorboard
%reload_ext tensorboard
%tensorboard --logdir classification_results/categorization/ResNet_Numeric-F_BatchSize-8_Epochs-20_Norm-T/logs

In [ ]:
# EVALUATE MODEL
print("Loaded best weights of the training")
model.load_weights(save_dir + "best_model.h5")

results = model.evaluate(test_dataset, 
                         steps = len(test_dataset), 
                         verbose = 1)

In [ ]:
# MAKE PREDICTIONS
# Retrieve a batch of images from the test set
predictions = model.predict(test_dataset.)

predictions = tf.where(predictions < 0.5, 0, 1)

print('Predictions:\n', predictions.numpy())
print('Labels:\n')
for label in test_labels_dataset:
    print(label)
#class_names = {0: "No eye", 1: "Eye"}
#plt.figure(figsize=(10, 10))
#for i in range(9):
#  ax = plt.subplot(3, 3, i + 1)
#  plt.imshow(image_batch[i].astype("uint8"))
#  plt.title(class_names[predictions[i]])
#  plt.axis("off")

## 2.Train on data using the Stratified K Fold

In [ ]:
# STRATIFIED CROSS VALIDATION
def fold_cross_validation(main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, EYE_ONLY, NUM_VARS, NORMALISE, NORM_MODE, ROTATE, N_CROPS, 
                          CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE):

    full_dataset_path = "{}/csv/full_dataset.csv".format(main_dir)
    df = pd.read_csv(full_dataset_path, converters={'bbox_shape': eval}).dropna()
    print("Dataset dimension: {}".format(len(df)))

    # COMPUTE CLASS WEIGHTS
    # if CROP_MODE == weighted (proporional to presence of classes), helper.compute_class_weights()
    # will no longer represent the true distribution of classes in the dataset
    class_weights = helper.compute_class_weights(full_dataset_path, eye_only = EYE_ONLY) if CROP_MODE != "weighted" else None

    # extract non-categorical labels for StratifiedKFold
    #Y = df["label"]
    Y = np.zeros(len(df), dtype=int)
    for index, row in df.iterrows():
        if df["label"][index] != 0:
            cat = df["image"][index].split('/')[1]
            Y[index] = int(cat[-1])
    
    if EYE_ONLY:
        assert isinstance(Y, np.ndarray)
        idx = np.where(Y == 0)[0].tolist()
        Y = np.delete(Y, idx)
        Y -= 1
        df.drop(idx, inplace = True)
        df.reset_index(drop=True, inplace=True)
        #print(df)
        print("New dimensions, Y: {} and df: {}".format(len(Y), len(df)))

    VALIDATION_ACCURACY = []
    VALIDATION_TOP2_ACCURACY = []
    VALIDATION_LOSS = []
    VALIDATION_TP = []
    VALIDATION_FP = []
    VALIDATION_TN = []
    VALIDATION_FN = []
    VALIDATION_PRECISION = []
    VALIDATION_RECALL = []

    # directory to save results
    dir = '{}_nu-{}_bs-{}_{}x{}_lr-{}_ep-{}_sp-{}_no-{}{}_cr-{}{}_ag-{}_drp-{}{}'.format(
        NETWORK, str(NUM_VARS)[0], BATCH_SIZE, MIN_HEIGHT, MIN_WIDTH, LEARNING_RATE, EPOCHS, SPLIT_NUMBER, 
        NORM_MODE[0], str(NORMALISE)[0], CROP_MODE[0], N_CROPS, str(AUGMENT)[0], str(DROPOUT)[0], DROP_RATE)
    save_dir = main_dir + '/classification_results/categorization/test_eye_only/' + dir + '/'


    # create an instance of the DataProcessor
    processor = dp.DataProcessor(model = NETWORK,
                                min_height = MIN_HEIGHT,
                                min_width = MIN_WIDTH,
                                normalise = NORMALISE,           # perform normalisation [DEPRECATED]
                                rotate = ROTATE,                 # perform rotation
                                plot_light = False,              # plot only select_crop() images
                                plot_extensive = False,          # plot extensively all images
                                show_prints = False)


    print("Entering in k fold cross validation...")
    stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, shuffle = False)
    fold_var = 1

    for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
        training_data = df.iloc[train_index]
        validation_data = df.iloc[val_index]

        # LOAD DATA
        train_images, train_labels, train_bbox = helper.load_data(df = training_data, eye_only = EYE_ONLY)
        #print("Train:", len(train_images), len(train_labels), len(train_bbox))
        val_images, val_labels, val_bbox = helper.load_data(df = validation_data, eye_only = EYE_ONLY)
        #print("Validation:", len(val_images), len(val_labels), len(val_bbox))

        # GENERATE DATASETS
        #train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
        #val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
        train_dataset = helper.create_dataset(processor, train_images, train_labels, train_bbox, N_CROPS, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
        val_dataset = helper.create_dataset(processor, val_images, val_labels, val_bbox, 1, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
        # since we increased dataset size with more crops, val_labels also increased
        #val_labels = val_dataset.map(lambda x, y: y)

        # PERFORM NORMALISATION
        if NORMALISE:
            train_norm_dataset, val_norm_dataset = helper.normalisation(train_dataset, val_dataset, mode = NORM_MODE, model = NETWORK)
        else:
            train_norm_dataset = train_dataset
            val_norm_dataset = val_dataset
        unbatched_val_norm_dataset = val_norm_dataset

        # CONFIGURE FOR PERFORMANCE
        SQUARED = True if MIN_HEIGHT == MIN_WIDTH else False
        train_norm_dataset = helper.configure_for_performance(train_norm_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True, augment = AUGMENT, squared_input = SQUARED)
        val_norm_dataset = helper.configure_for_performance(val_norm_dataset, BUFFER_SIZE, BATCH_SIZE)

        # CREATE NEW MODEL
        model = models.make_cnn(NETWORK, MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, eye_only = EYE_ONLY, dropout = DROPOUT, rate = DROP_RATE)
        #if NETWORK == "Mobile":
        #    model = models.make_MobileNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, eye_only = True)
        #elif NETWORK == "ResNet":
        #    model = models.make_ResNet(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, eye_only = True)
        #elif NETWORK == "VGG":
        #    model = models.make_VGG(MIN_WIDTH, MIN_HEIGHT, LEARNING_RATE, eye_only = True)
        #else:
        #    sys.exit("Incert valid network model. Options: Mobile, ResNet or VGG (case sensitive)")

        # CREATE CALLBACKS
        callbacks = [
            #EarlyStopping(patience = 10, verbose = 1),
            ReduceLROnPlateau(factor = 0.1, patience = 5, min_lr = 0.000001, verbose = 1),
            ModelCheckpoint(save_dir + helper.get_model_name(fold_var), verbose = 1, save_best_only = True),
            #TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
        ]

        # FIT THE MODEL
        history = model.fit(
            train_norm_dataset,
            steps_per_epoch = len(train_norm_dataset),
            validation_data = val_norm_dataset,
            validation_steps = len(val_norm_dataset), 
            epochs = EPOCHS,
            callbacks = callbacks,
            verbose = 1, 
            class_weight = class_weights,
            shuffle = True)
        
        # PLOT HISTORY
        #print(history)


        # PLOT TRAIN/VALIDATION LOSSES
        fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, figsize=(10, 10))
        ax1.plot(history.history['accuracy'], label='Training Accuracy')
        ax1.plot(history.history['val_accuracy'], label='Validation Accuracy')
        ax1.grid(True)
        ax1.legend(loc='lower right')
        ax1.set(ylabel = "Accuracy",
                title = 'Training and Validation Accuracy')

        ax2.plot(history.history["loss"], label='Training Loss')
        ax2.plot(history.history["val_loss"], label='Validation Loss')
        ax2.plot(np.argmin(history.history["val_loss"]), np.min(history.history["val_loss"]), marker="x", color="r", label = "Best model")
        ax2.grid(True)
        ax2.legend(loc='upper right')
        ax2.set(xlabel = 'Epoch', 
                ylabel = 'Cross Entropy',
                title = 'Training and Validation Loss')
        plt.show()
        fig.savefig(save_dir + "model_" + str(fold_var) + ".jpg", bbox_inches='tight')

        
        # LOAD BEST MODEL
        time.sleep(10) # guarantees enough time for weights to be saved and loaded afterwards, otherwise gives concurrency problems
        print("Loaded best weights of the training")
        model.load_weights(save_dir + helper.get_model_name(fold_var))

        # EVALUATE PERFORMANCE of the model
        results = model.evaluate(val_norm_dataset,
                                steps = len(val_norm_dataset), 
                                verbose = 1)

        results = dict(zip(model.metrics_names, results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_TOP2_ACCURACY.append(results['top2_accuracy'])
        VALIDATION_LOSS.append(results['loss'])
        VALIDATION_TP.append(results["tp"])
        VALIDATION_FP.append(results["fp"])
        VALIDATION_TN.append(results["tn"])
        VALIDATION_FN.append(results["fn"])
        VALIDATION_PRECISION.append(results["precision"])
        VALIDATION_RECALL.append(results["recall"])

        # MAKE PREDICTIONS
        predictions = model.predict(val_norm_dataset)
        predictions_non_category = [ np.argmax(t) for t in predictions ]
        val_labels_non_category = [ np.argmax(t) for t in val_labels ]

        labels = [0,1,2,3,4,5] if not EYE_ONLY else [0,1,2,3,4]
        display_labels = labels if not EYE_ONLY else [x+1 for x in labels]
        conf_mat = confusion_matrix(val_labels_non_category, predictions_non_category, labels = labels)
        disp = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = display_labels)
        conf_mat_display = disp.plot()
        plt.savefig(save_dir + "confusion_matrix_" + str(fold_var) + ".jpg", bbox_inches='tight')

        # GRAD-CAM ANALYSIS
        gradcam_path = save_dir + "gradcam_heatmaps_{}".format(fold_var)
        os.makedirs(gradcam_path, exist_ok=True)
        helper.grad_cam(model, val_dataset, unbatched_val_norm_dataset, predictions, gradcam_path, dropout = DROPOUT)

        tf.keras.backend.clear_session()

        fold_var += 1

        # save the values for each fold
        csv_dir = save_dir + 'csv/'
        os.makedirs(csv_dir, exist_ok=True)
        with open(csv_dir + 'test_accuracy.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_ACCURACY, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_top2_accuracy.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TOP2_ACCURACY, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_loss.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_LOSS, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_tp.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TP, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_fp.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_FP, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_tn.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TN, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_fn.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_FN, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_precision.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_PRECISION, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_recall.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_RECALL, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    return

###2.1. Run tests

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS_dilated"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet", "VGG"]
#MIN_HEIGHT = 288
#MIN_WIDTH = 288
EYE_ONLY = True
NUM_VARS = False
NORMALISE = True
#NORM_MODE = "model" # options: ["model", "z-norm", "simple", "none"]
ROTATE = False
#N_CROPS = 3
CROP_MODE = "uniform" # options: ["uniform", "weighted"]
AUGMENT = True
BATCH_SIZE = 8
BUFFER_SIZE = 100
EPOCHS = 30
LEARNING_RATE = 0.00001
SPLIT_NUMBER = 5
DROPOUT = True
DROP_RATE = 0.8
##################################

# Order of inputs:
# main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, EYE_ONLY, NUM_VARS, NORMALISE, NORM_MODE, ROTATE, N_CROPS, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE

#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 288, 288, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 352, 352, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 416, 416, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
fold_cross_validation(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 416, 416, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)
#fold_cross_validation(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, False, "none", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, EPOCHS, LEARNING_RATE, SPLIT_NUMBER, DROPOUT, DROP_RATE)

## 3.Perform Fine-Tuning

### 3.1. Using the csv split into train, val and test sets.

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet"]
MIN_HEIGHT = 700
MIN_WIDTH = 400
NUM_VARS = False
NORMALISE = True
ROTATE = False
BATCH_SIZE = 8
BUFFER_SIZE = 100
EPOCHS = 50
LEARNING_RATE = 0.0001
##################################

# load data
train_images, train_labels, train_bbox = helper.load_data("{}/csv/training.csv".format(main_dir))
val_images, val_labels, val_bbox = helper.load_data("{}/csv/val.csv".format(main_dir))
test_images, test_labels, test_bbox = helper.load_data("{}/csv/test.csv".format(main_dir))
class_weights = helper.compute_class_weights("{}/csv/full_dataset.csv".format(main_dir))


# create an instance of the DataProcessor
processor = dp.DataProcessor(model = NETWORK,
                             min_height = MIN_HEIGHT,
                             min_width = MIN_WIDTH,
                             normalise = NORMALISE,           # perform normalisation
                             rotate = ROTATE,                 # perform rotation
                             plot_light = False,              # plot only select_crop() images
                             plot_extensive = False,          # plot extensively all images
                             show_prints = False)


# generate datasets
train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
test_dataset = helper.prepare_dataset(processor, test_images, test_labels, test_bbox)
#for image, label in train_dataset:
#  print("FINAL - image: {}, max: {}, min: {}, label: {}".format(image[0].shape, np.max(image[0]), np.min(image[0]),  label))


# configure for performance
train_dataset = helper.configure_for_performance(train_dataset, BUFFER_SIZE, BATCH_SIZE)
val_dataset = helper.configure_for_performance(val_dataset, BUFFER_SIZE, BATCH_SIZE)
test_dataset = helper.configure_for_performance(test_dataset, BUFFER_SIZE, BATCH_SIZE)

In [ ]:
# create the base pre-trained model
if NETWORK == "ResNet":
    base_model = models.ResNet50(weights='imagenet', include_top=False, input_shape=(MIN_WIDTH, MIN_HEIGHT, 3))
elif NETWORK == "Mobile":
    base_model = models.MobileNetV2(weights='imagenet', include_top=False, input_shape=(MIN_WIDTH, MIN_HEIGHT, 3))
else:
  sys.exit("Incert valid network model. Options: Mobile or ResNet (case sensitive)")

# Freeze the base_model
base_model.trainable = False

inputs = Input(shape=(MIN_WIDTH, MIN_HEIGHT, 3))
#x = data_augmentation(inputs)  # apply random data augmentation
x = base_model(inputs, training = False)
x = GlobalAveragePooling2D()(x)
outputs = Dense(6, activation="softmax")(x)
model = Model(inputs, outputs)

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer = Adam(learning_rate = LEARNING_RATE), 
              loss = "categorical_crossentropy",
              metrics = [CategoricalAccuracy(name="accuracy"), 
                          Precision(name="precision"), Recall(name="recall"), 
                          TruePositives(name='tp'), FalsePositives(name='fp'),
                          TrueNegatives(name='tn'), FalseNegatives(name='fn')])

model.summary()

In [ ]:
# directory to save results
dir = '{}_Numeric-{}_BatchSize-{}_lr-{}_Epochs-{}_Norm-{}_FineTune-T'.format(NETWORK, str(NUM_VARS)[0], str(BATCH_SIZE), str(LEARNING_RATE), str(EPOCHS), str(NORMALISE)[0])
save_dir = main_dir + '/classification_results/categorization/' + dir + '/'
os.makedirs(save_dir, exist_ok=True)

# CREATE CALLBACKS
callbacks = [
    ModelCheckpoint(save_dir + "best_model_frozen.h5", verbose = 1, save_best_only = True),
    TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
]

initial_epochs = 40

# train the top layer of the model on the dataset, the weights
# of the pre-trained network will not be updated during training
history = model.fit(train_dataset,
                    steps_per_epoch = len(train_dataset),
                    validation_data = val_dataset,
                    validation_steps = len(val_dataset),
                    epochs = initial_epochs,
                    callbacks = callbacks,
                    class_weight = class_weights,
                    shuffle = True)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, figsize=(10, 10))
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
ax1.legend(loc='lower right')
ax1.set(ylabel = "Accuracy",
        title = 'Training and Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])

ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
ax2.legend(loc='upper right')
ax2.set(xlabel = 'epoch', 
        ylabel = 'Cross Entropy',
        title = 'Training and Validation Loss')
#plt.ylim([0,max(plt.ylim())])

plt.show()

In [ ]:
# unfreeze the whole base model
base_model.trainable = True

# you should try to fine-tune a small number of top layers rather than the whole model
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# recompile the model for the modifications to take effect, with a low learning rate
model.compile(optimizer = Adam(learning_rate = 1e-5),
              loss = "categorical_crossentropy",
              metrics = [CategoricalAccuracy(name="accuracy"), 
                          Precision(name="precision"), Recall(name="recall"), 
                          TruePositives(name='tp'), FalsePositives(name='fp'),
                          TrueNegatives(name='tn'), FalseNegatives(name='fn')])

model.summary()

In [ ]:
# adjust callbacks
callbacks = [
    #EarlyStopping(patience = 5, verbose = 1),
    ReduceLROnPlateau(factor = 0.1, patience = 5, min_lr = 0.000001, verbose = 1),
    ModelCheckpoint(save_dir + "best_model_fine_tuned.h5", verbose = 1, save_best_only = True),
    TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
]

fine_tune_epochs = 10
total_epochs = initial_epochs + fine_tune_epochs

# train the entire model end-to-end
history_fine = model.fit(train_dataset,
                         steps_per_epoch = len(train_dataset),
                         validation_data = val_dataset,
                         validation_steps = len(val_dataset),
                         epochs = total_epochs,
                         initial_epoch = history.epoch[-1],
                         callbacks = callbacks,
                         class_weight = class_weights,
                         shuffle = True)

In [ ]:
acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']

loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']


fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, figsize=(10, 10))
ax1.plot(acc, label='Training Accuracy')
ax1.plot(val_acc, label='Validation Accuracy')
#plt.ylim([min(plt.ylim()),1])
ax1.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
ax1.legend(loc='lower right')
ax1.set(ylabel = "Accuracy",
        title = 'Training and Validation Accuracy')

ax2.plot(loss, label='Training Loss')
ax2.plot(val_loss, label='Validation Loss')
#plt.ylim([0, 1.0])
ax2.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
ax2.legend(loc='upper right')
ax2.set(xlabel = 'epoch', 
      ylabel = 'Cross Entropy',
      title = 'Training and Validation Loss')
plt.show()

fig.savefig(save_dir + 'Learning_curves.png', bbox_inches='tight')

In [ ]:
# EVALUATE MODEL
print("Loaded best weights of the training")
model.load_weights(save_dir + "best_model_fine_tuned.h5")

results = model.evaluate(test_dataset, 
                         steps = len(test_dataset), 
                         verbose = 1)

### 3.2. Using Stratified-K Fold.

In [ ]:
# STRATIFIED CROSS VALIDATION WITH TRANSFER LEARNING AND FINE-TUNING
def fold_cross_validation_ft(main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, EYE_ONLY, NUM_VARS, NORMALISE, NORM_MODE, ROTATE, N_CROPS, CROP_MODE, AUGMENT, 
                             BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE):

    full_dataset_path = "{}/csv/full_dataset.csv".format(main_dir)
    df = pd.read_csv(full_dataset_path, converters={'bbox_shape': eval}).dropna()
    print("Dataset dimension: {}".format(len(df)))

    # COMPUTE CLASS WEIGHTS
    # if CROP_MODE == weighted (proporional to presence of classes), helper.compute_class_weights()
    # will no longer represent the true distribution of classes in the dataset
    class_weights = helper.compute_class_weights(full_dataset_path, eye_only = EYE_ONLY) if CROP_MODE != "weighted" else None

    # extract non-categorical labels for StratifiedKFold
    #Y = df["label"]
    Y = np.zeros(len(df), dtype=int)
    for index, row in df.iterrows():
        if df["label"][index] != 0:
            cat = df["image"][index].split('/')[1]
            Y[index] = int(cat[-1])

    if EYE_ONLY:
        assert isinstance(Y, np.ndarray)
        idx = np.where(Y == 0)[0].tolist()
        Y = np.delete(Y, idx)
        Y -= 1
        df.drop(idx, inplace = True)
        df.reset_index(drop=True, inplace=True)
        #print(df)
        print("New dimensions, Y: {} and df: {}".format(len(Y), len(df)))
    
    VALIDATION_ACCURACY = []
    VALIDATION_TOP2_ACCURACY = []
    VALIDATION_LOSS = []
    VALIDATION_TP = []
    VALIDATION_FP = []
    VALIDATION_TN = []
    VALIDATION_FN = []
    VALIDATION_PRECISION = []
    VALIDATION_RECALL = []

    # directory to save results
    total_epochs = INITIAL_EPOCHS + FINE_TUNE_EPOCHS
    dir = '{}_nu-{}_bs-{}_{}x{}_lr-{}_ep-{}_sp-{}_no-{}{}_cr-{}{}_ag-{}_drp-{}{}_ft-{}'.format(
        NETWORK, str(NUM_VARS)[0], BATCH_SIZE, MIN_HEIGHT, MIN_WIDTH, LEARNING_RATE, total_epochs, SPLIT_NUMBER, 
        NORM_MODE[0], str(NORMALISE)[0], CROP_MODE[0], N_CROPS, str(AUGMENT)[0], str(DROPOUT)[0], DROP_RATE, FINE_TUNE_AT)
    save_dir = main_dir + '/classification_results/categorization/test_eye_only/' + dir + '/'

    
    # create an instance of the DataProcessor
    processor = dp.DataProcessor(model = NETWORK,
                                 min_height = MIN_HEIGHT,
                                 min_width = MIN_WIDTH,
                                 normalise = NORMALISE,           # perform normalisation
                                 rotate = ROTATE,                 # perform rotation
                                 plot_light = False,              # plot only select_crop() images
                                 plot_extensive = False,          # plot extensively all images
                                 show_prints = False)


    print("Entering in k fold cross validation...")
    stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, shuffle = False)    
    fold_var = 1

    for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
        training_data = df.iloc[train_index]
        validation_data = df.iloc[val_index]

        # LOAD DATA
        train_images, train_labels, train_bbox = helper.load_data(df = training_data, eye_only = EYE_ONLY)
        val_images, val_labels, val_bbox = helper.load_data(df = validation_data, eye_only = EYE_ONLY)
        
        # GENERATE DATASETS
        #train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
        #val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
        train_dataset = helper.create_dataset(processor, train_images, train_labels, train_bbox, N_CROPS, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
        val_dataset = helper.create_dataset(processor, val_images, val_labels, val_bbox, 1, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)

        # PERFORM NORMALISATION
        if NORMALISE:
            train_norm_dataset, val_norm_dataset = helper.normalisation(train_dataset, val_dataset, mode = NORM_MODE, model = NETWORK)
        else:
            train_norm_dataset = train_dataset
            val_norm_dataset = val_dataset
        unbatched_val_norm_dataset = val_norm_dataset

        # CONFIGURE FOR PERFORMANCE
        SQUARED = True if MIN_HEIGHT == MIN_WIDTH else False
        train_norm_dataset = helper.configure_for_performance(train_norm_dataset, BUFFER_SIZE, BATCH_SIZE, shuffle = True, augment = AUGMENT, squared_input = SQUARED)
        val_norm_dataset = helper.configure_for_performance(val_norm_dataset, BUFFER_SIZE, BATCH_SIZE)
        
        # CREATE BASE PRE-TRAINED MODEL
        if NETWORK == "ResNet":
            base_model = models.ResNet50(weights='imagenet', include_top=False, input_shape=(MIN_WIDTH, MIN_HEIGHT, 3))
        elif NETWORK == "Mobile":
            base_model = models.MobileNetV2(weights='imagenet', include_top=False, input_shape=(MIN_WIDTH, MIN_HEIGHT, 3))
        else:
            sys.exit("Incert valid network model. Options: Mobile or ResNet (case sensitive)")

        # FREEZE BASE MODEL
        base_model.trainable = False

        inputs = Input(shape=(MIN_WIDTH, MIN_HEIGHT, 3))
        x = base_model(inputs, training = False)
        x = GlobalAveragePooling2D()(x)
        if DROPOUT:
            x = Dropout(DROP_RATE)(x)
        classes = 6 if not EYE_ONLY else 5
        outputs = Dense(classes, activation="softmax")(x)
        model = Model(inputs, outputs)

        # compile the model (should be done *after* setting layers to non-trainable)
        model.compile(optimizer = Adam(learning_rate = LEARNING_RATE), 
                      loss = "categorical_crossentropy",
                      metrics = [CategoricalAccuracy(name="accuracy"), TopKCategoricalAccuracy(k=2, name="top2_accuracy"),
                                 Precision(name="precision"), Recall(name="recall"), 
                                 TruePositives(name='tp'), FalsePositives(name='fp'),
                                 TrueNegatives(name='tn'), FalseNegatives(name='fn')])

        # CREATE CALLBACKS
        callbacks = [
            ModelCheckpoint(save_dir + "best_model_frozen_{}.h5".format(str(fold_var)), verbose = 1, save_best_only = True),
            #TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
        ]

        # FIT THE MODEL
        # train the top layer of the model on the dataset, the weights
        # of the pre-trained network will not be updated during training
        history = model.fit(train_norm_dataset,
                            steps_per_epoch = len(train_norm_dataset),
                            validation_data = val_norm_dataset,
                            validation_steps = len(val_norm_dataset),
                            epochs = INITIAL_EPOCHS,
                            callbacks = callbacks,
                            class_weight = class_weights,
                            shuffle = True)

        acc = history.history['accuracy']
        val_acc = history.history['val_accuracy']

        loss = history.history['loss']
        val_loss = history.history['val_loss']

        # UNFREEZE BASE MODEL
        base_model.trainable = True

        # Fine-tune from this layer onwards
        if isinstance(FINE_TUNE_AT, int):
            # Freeze all the layers before the 'FINE_TUNE_AT' layer
            for layer in base_model.layers[:FINE_TUNE_AT]:
                layer.trainable = False
            
        else: #string
            if "last" in FINE_TUNE_AT:
                nb_layers_to_fine_tune = int(re.findall(r'\d+', FINE_TUNE_AT)[0])
                print("nb_layers_to_fine_tune:", nb_layers_to_fine_tune)
                total_to_freeze = len(base_model.layers) - nb_layers_to_fine_tune
                print("total_to_freeze:", total_to_freeze)
                for layer in base_model.layers[:total_to_freeze]:
                    layer.trainable = False


        # recompile the model for the modifications to take effect, with a low learning rate
        model.compile(optimizer = Adam(learning_rate = 1e-5),
                      loss = "categorical_crossentropy",
                      metrics = [CategoricalAccuracy(name="accuracy"), TopKCategoricalAccuracy(k=2, name="top2_accuracy"),
                                 Precision(name="precision"), Recall(name="recall"), 
                                 TruePositives(name='tp'), FalsePositives(name='fp'),
                                 TrueNegatives(name='tn'), FalseNegatives(name='fn')])
        #base_model.summary()
        
        # adjust callbacks
        callbacks = [
            ModelCheckpoint(save_dir + "best_model_fine_tuned_{}.h5".format(str(fold_var)), verbose = 1, save_best_only = True),
            #TensorBoard(log_dir = save_dir + "logs/" + datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))
        ]

        # train the entire model end-to-end
        history_fine = model.fit(train_norm_dataset,
                                 steps_per_epoch = len(train_norm_dataset),
                                 validation_data = val_norm_dataset,
                                 validation_steps = len(val_norm_dataset),
                                 epochs = total_epochs,
                                 initial_epoch = history.epoch[-1],
                                 callbacks = callbacks,
                                 class_weight = class_weights,
                                 shuffle = True)

        acc += history_fine.history['accuracy']
        val_acc += history_fine.history['val_accuracy']

        loss += history_fine.history['loss']
        val_loss += history_fine.history['val_loss']

        # PLOT TRAIN/VALIDATION LOSSES
        fig, (ax1, ax2) = plt.subplots(nrows = 2, ncols = 1, figsize=(10, 10))
        ax1.plot(acc, label='Training Accuracy')
        ax1.plot(val_acc, label='Validation Accuracy')
        #plt.ylim([min(plt.ylim()),1])
        ax1.plot([INITIAL_EPOCHS-1, INITIAL_EPOCHS-1], plt.ylim(), label='Start Fine Tuning')
        ax1.grid(True)
        ax1.legend(loc='lower right')
        ax1.set(ylabel = "Accuracy",
                title = 'Training and Validation Accuracy')

        ax2.plot(loss, label='Training Loss')
        ax2.plot(val_loss, label='Validation Loss')
        #plt.ylim([0, 1.0])
        ax2.plot([INITIAL_EPOCHS-1, INITIAL_EPOCHS-1], plt.ylim(), label='Start Fine Tuning')
        ax2.plot(np.argmin(val_loss), np.min(val_loss), marker="x", color="r", label = "Best model")
        ax2.grid(True)
        ax2.legend(loc='upper right')
        ax2.set(xlabel = 'Epoch',
                ylabel = 'Cross Entropy',
                title = 'Training and Validation Loss')
        plt.show()
        fig.savefig(save_dir + "model_" + str(fold_var) + ".jpg", bbox_inches='tight')
        
        ##################################################################################################
        

        # LOAD BEST MODEL
        time.sleep(15) # guarantees enough time for weights to be saved and loaded afterwards, otherwise gives concurrency problems
        print("Loaded best weights of the training")
        model.load_weights(save_dir + "best_model_fine_tuned_{}.h5".format(str(fold_var)))

        # EVALUATE PERFORMANCE of the model
        results = model.evaluate(val_norm_dataset, 
                                 steps = len(val_norm_dataset), 
                                 verbose = 1)

        results = dict(zip(model.metrics_names, results))

        VALIDATION_ACCURACY.append(results['accuracy'])
        VALIDATION_TOP2_ACCURACY.append(results['top2_accuracy'])
        VALIDATION_LOSS.append(results['loss'])
        VALIDATION_TP.append(results["tp"])
        VALIDATION_FP.append(results["fp"])
        VALIDATION_TN.append(results["tn"])
        VALIDATION_FN.append(results["fn"])
        VALIDATION_PRECISION.append(results["precision"])
        VALIDATION_RECALL.append(results["recall"])

        # MAKE PREDICTIONS
        predictions = model.predict(val_norm_dataset)
        predictions_non_category = [ np.argmax(t) for t in predictions ]
        val_labels_non_category = [ np.argmax(t) for t in val_labels ]

        labels = [0,1,2,3,4,5] if not EYE_ONLY else [0,1,2,3,4]
        display_labels = labels if not EYE_ONLY else [x+1 for x in labels]
        conf_mat = confusion_matrix(val_labels_non_category, predictions_non_category, labels = labels)
        disp = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = display_labels)
        conf_mat_display = disp.plot()
        plt.savefig(save_dir + "confusion_matrix_" + str(fold_var) + ".jpg", bbox_inches='tight')

        # GRAD-CAM ANALYSIS
        gradcam_path = save_dir + "gradcam_heatmaps_{}".format(fold_var)
        os.makedirs(gradcam_path, exist_ok=True)
        helper.grad_cam(model, val_dataset, unbatched_val_norm_dataset, predictions, gradcam_path, dropout = DROPOUT, fine_tuning = True, network = NETWORK)

        tf.keras.backend.clear_session()

        fold_var += 1

        # save the values for each fold
        csv_dir = save_dir + 'csv/'
        os.makedirs(csv_dir, exist_ok=True)
        with open(csv_dir + 'test_accuracy.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_ACCURACY, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_top2_accuracy.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TOP2_ACCURACY, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_loss.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_LOSS, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_tp.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TP, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_fp.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_FP, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_tn.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_TN, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_fn.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_FN, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_precision.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_PRECISION, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(csv_dir + 'test_recall.pkl', 'wb') as handle:
            pickle.dump(VALIDATION_RECALL, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return

####3.2.1. Run tests

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS_dilated"
NETWORK = "ResNet"     # options: ["Mobile", "ResNet"]
#MIN_HEIGHT = 288
#MIN_WIDTH = 288
EYE_ONLY = True
NUM_VARS = False
NORMALISE = True
#NORM_MODE = "model" # options: ["model", "z-norm", "simple"]
ROTATE = False
#N_CROPS = 3
CROP_MODE = "uniform" # options: ["uniform", "weighted"]
AUGMENT = True
BATCH_SIZE = 8
BUFFER_SIZE = 100
INITIAL_EPOCHS = 30
FINE_TUNE_EPOCHS = 10
LEARNING_RATE = 0.0001
SPLIT_NUMBER = 5
FINE_TUNE_AT = "last5"  # options: int() - Layer to start fine-tuning from;
                        #          str() - Number of last layers to fine-tune: "lastX"
DROPOUT = True
DROP_RATE = 0.5                     
##################################

# Order of inputs:
#main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, EYE_ONLY, NUM_VARS, NORMALISE, NORM_MODE, ROTATE, N_CROPS, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE

# Tests:
#fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)
#fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)
#fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "model", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)
fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)
#fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 3, CROP_MODE, AUGMENT, BATCH_SIZE, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)
#fold_cross_validation_ft(main_dir, NETWORK, 384, 384, EYE_ONLY, NUM_VARS, NORMALISE, "z-norm", ROTATE, 5, CROP_MODE, AUGMENT, 16, BUFFER_SIZE, INITIAL_EPOCHS, FINE_TUNE_EPOCHS, LEARNING_RATE, SPLIT_NUMBER, FINE_TUNE_AT, DROPOUT, DROP_RATE)

## 4.Test random things

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

val_labels1 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5]
predictions1 =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,2,2,3,1,1,1,3,4,0,3,3,3,4,3,3,4,4,4,4,5]

val_labels2 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,3,4,4,4,4,4,5,5]
predictions2 =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,1,2,2,2,3,1,1,2,3,3,3,1,1,2,2,3,2,4]

val_labels3 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,4,5]
predictions3 =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,1,1,1,1,2,2,2,3,4,4,0,0,0,1,2,1,2,2,3,4,1,4,4,4,5,5,4]

val_labels4 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5]
predictions4 =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1,1,1,1,1,1,1,2,3,3,1,1,1,3,3,2,2,3,4,4,1,1,4,4,4,0,3]

val_labels5 = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,2,2,2,2,2,3,3,3,3,3,4,4,4,4,4,5,5]
predictions5 =[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,3,4,0,1,1,1,3,1,1,3,3,3,1,3,4,4,4,0,3]


val_labels = val_labels1 + val_labels2 + val_labels3 + val_labels4 + val_labels5
predictions = predictions1 + predictions2 + predictions3 + predictions4 + predictions5

conf_mat = confusion_matrix(val_labels5, predictions5, labels = [0,1,2,3,4,5])
disp = ConfusionMatrixDisplay(confusion_matrix = conf_mat, display_labels = [0,1,2,3,4,5])
conf_mat_display = disp.plot()

import matplotlib.pyplot as plt
#plt.savefig("SAR_swath_images_VV+VH+WS/classification_results/categorization/ResNet_Numeric-F_BatchSize-8_lr-0.0001_Epochs-20_Folds-5_Norm-T_Aug-T/global_confusion_matrix.jpg")

In [ ]:
##################################
###          SETTINGS         ####
##################################
main_dir = "SAR_swath_images_VV+VH+WS_dilated"
NETWORK = "Mobile"     # options: ["Mobile", "ResNet"]
#MIN_HEIGHT = 288
#MIN_WIDTH = 288
#NORM_MODE = "model" # options: ["model", "z-norm", "simple"]
#N_CROPS = 3

CROP_MODE = "uniform" # options: ["uniform", "weighted"]
NUM_VARS = False
NORMALISE = True
ROTATE = False
AUGMENT = True
SPLIT_NUMBER = 5
##################################


def test_normalisation_modes(main_dir, NETWORK, MIN_HEIGHT, MIN_WIDTH, NUM_VARS, NORMALISE, NORM_MODE, ROTATE, N_CROPS, 
                          CROP_MODE, AUGMENT, SPLIT_NUMBER):
    
    full_dataset_path = "{}/csv/full_dataset.csv".format(main_dir)
    df = pd.read_csv(full_dataset_path, converters={'bbox_shape': eval}).dropna()
    print("Dataset dimension: {}".format(len(df)))

    # COMPUTE CLASS WEIGHTS
    # if CROP_MODE == weighted (proporional to presence of classes), helper.compute_class_weights()
    # will no longer represent the true distribution of classes in the dataset
    class_weights = helper.compute_class_weights(full_dataset_path) if CROP_MODE != "weighted" else None

    # extract non-categorical labels for StratifiedKFold
    #Y = df["label"]
    Y = np.zeros(len(df), dtype=int)
    for index, row in df.iterrows():
        if df["label"][index] != 0:
            cat = df["image"][index].split('/')[1]
            Y[index] = int(cat[-1])


    # create an instance of the DataProcessor
    processor = dp.DataProcessor(model = NETWORK,
                                min_height = MIN_HEIGHT,
                                min_width = MIN_WIDTH,
                                normalise = NORMALISE,           # perform normalisation [DEPRECATED]
                                rotate = ROTATE,                 # perform rotation
                                plot_light = False,              # plot only select_crop() images
                                plot_extensive = False,          # plot extensively all images
                                show_prints = False)


    print("Entering in k fold cross validation...")
    stratified_k_fold = StratifiedKFold(n_splits = SPLIT_NUMBER, shuffle = False)
    fold_var = 1

    for train_index, val_index in stratified_k_fold.split(np.zeros(len(df)), Y):
        training_data = df.iloc[train_index]
        validation_data = df.iloc[val_index]

        # LOAD DATA
        train_images, train_labels, train_bbox = helper.load_data(df = training_data)
        val_images, val_labels, val_bbox = helper.load_data(df = validation_data)
        
        # GENERATE DATASETS
        #train_dataset = helper.prepare_dataset(processor, train_images, train_labels, train_bbox)
        #val_dataset = helper.prepare_dataset(processor, val_images, val_labels, val_bbox)
        train_dataset = helper.create_dataset(processor, train_images, train_labels, train_bbox, N_CROPS, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
        val_dataset = helper.create_dataset(processor, val_images, val_labels, val_bbox, 1, CROP_MODE, MIN_HEIGHT, MIN_WIDTH)
        # since we increased dataset size with more crops, val_labels also increased
        #val_labels = val_dataset.map(lambda x, y: y)

        # PERFORM NORMALISATION
        if NORMALISE:
            train_norm_dataset, val_norm_dataset = helper.normalisation(train_dataset, val_dataset, mode = NORM_MODE, model = NETWORK)

        train_norm_images = train_norm_dataset.map(lambda x, y: x)
        train_images = train_dataset.map(lambda x, y: x)

        cnt = 0
        dir = "{}/model_norm_2/{}".format(main_dir, fold_var)
        os.makedirs(dir, exist_ok=True)
        min = np.inf
        max = 0
        for norm_img, img in zip(train_norm_images, train_images):
            
            arr = norm_img.numpy()
            print("Norm image range:", np.max(arr), np.min(arr))
            #if np.max(arr) > max:
            #    max = np.max(arr)
            #if np.min(arr) < min:
            #    min = np.min(arr)
            #new_arr = ((arr - arr.min()) * (1/(arr.max() - arr.min()) * 255)).astype('uint8')
            #print(np.max(new_arr), np.min(new_arr))
            #imstack = np.hstack((img.numpy(), new_arr))
            #cv2_imshow(imstack)

            #print("------------------------------------------------------")
            #cnt += 1
            #cv2.imwrite("{}/image_{}.png".format(dir, cnt), imstack)
            #im = Image.fromarray(imstack, 'RGB')
            #im.save("{}/image_{}.png".format(dir, cnt))
            #plt.imshow(imstack)
            #plt.savefig("{}/image_{}.png".format(dir, cnt))
            #matplotlib.image.imsave("{}/image_{}.png".format(dir, cnt), imstack)

        fold_var += 1

        if fold_var > 1:
            print(min, max)
            import sys
            sys.exit()


test_normalisation_modes(main_dir, NETWORK, 288, 288, NUM_VARS, NORMALISE, "z-norm", ROTATE, 1, CROP_MODE, AUGMENT, SPLIT_NUMBER)

In [ ]:
zero = tf.constant(0, dtype = tf.float32)
print("zero:", zero)
nan = tf.constant(np.nan, dtype = tf.float32)
print("nan:", nan)
A = tf.convert_to_tensor(np.eye(6), dtype = tf.float32)
print("A:", A)
img = tf.where(tf.equal(A, zero), nan, A)
print("img:", img)

mean = 4
std = 3
aux = (img - mean)/std
print("aux:", aux)
new = tf.where(tf.math.is_nan(aux), tf.zeros_like(aux), aux)
print("new:", new)